In [104]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import undetected_chromedriver as uc

import time

import pandas as pd
import pickle

In [60]:
all_data = []

In [87]:
c_options = Options()
c_options.add_argument("--start-maximized")
c_options.add_argument("--disable-gpu")
c_options.add_argument("--no-sandbox")
c_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")

c_driver = uc.Chrome(options=c_options)

In [88]:
url = "https://www.milanuncios.com/coches-de-segunda-mano/?s=cupra"

c_driver.get(f'{url}&pagina=1')
time.sleep(10)

cookies = c_driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div/div/div[3]/button[3]')
cookies.click()


data = []
for i in range(2,12):
    
    scroll_height = c_driver.execute_script("return document.body.scrollHeight")

    # Perform smooth scrolling by incrementally scrolling down
    current_position = 0
    step_size = 500  # Number of pixels to scroll at a time 
    while current_position < scroll_height:
        c_driver.execute_script(f"window.scrollTo(0, {current_position});")
        time.sleep(0.75)  # Delay to simulate smoothness
        current_position += step_size
    
    articleNumber = 1
    SpanNumber = 1
    cars = c_driver.find_elements(By.TAG_NAME, 'article')
    for car in cars:
        carClass = car.get_attribute('class')
        if 'ma-AdCardV2' in carClass:
            try:
                car_type = car.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div[1]/div[2]/div[3]/div/main/div[3]/article[{articleNumber}]/div/div[1]/div/a/h2')
                price = car.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div[1]/div[2]/div[3]/div/main/div[3]/article[{articleNumber}]/div/div[1]/a[1]/div[1]/div/p/span[1]')
                km = car.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div[1]/div[2]/div[3]/div/main/div[3]/article[{articleNumber}]/div/div[1]/a[2]/div[2]/ul/li[1]/span/span')
                year = car.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div[1]/div[2]/div[3]/div/main/div[3]/article[{articleNumber}]/div/div[1]/a[2]/div[2]/ul/li[2]/span/span')
                description = car.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div[1]/div[2]/div[3]/div/main/div[3]/article[{articleNumber}]/div/div[1]/a[2]/div[3]/p').text
                photo = car.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div[1]/div[2]/div[3]/div/main/div[3]/article[{articleNumber}]/a/div/picture/img')
            except NoSuchElementException:
                car_type = False,
                price = False,
                km = False,
                year = False,
                description = False,
                photo = False
            
            if car_type and price and km and year and description and photo:
                priceClass =  price.get_attribute('class')
                if 'ma-AdPrice-value' in priceClass:
                    SpanNumber = 1
                else:
                    SpanNumber = 2

                car_type_text = car_type.text
                print(f'car_type: {car_type_text}')
                price_text = car.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div[1]/div[2]/div[3]/div/main/div[3]/article[{articleNumber}]/div/div[1]/a[1]/div[1]/div/p/span/span').text
                
                km_text = km.text
                
                year_text = year.text

                description_text = description
                photo_url = photo.get_attribute('src')

                all_data.append({
                        'car_type': car_type_text,
                        'year': year_text,
                        'price': price_text,
                        'km': km_text,
                        'description': description_text,
                        'photo_url': photo_url,
                    })
                
            articleNumber += 1
        elif 'ma-AdCardCarousel' in carClass:
            print('Ad de carrousel, no suma al article number')
        else:
            articleNumber += 1

    print(len(data),f'pagina: {i-1}')
    c_driver.get(f'{url}&pagina={i}')
    time.sleep(2)    


data

car_type: CUPRA - FORMENTOR
Ad de carrousel, no suma al article number
Ad de carrousel, no suma al article number
Ad de carrousel, no suma al article number
Ad de carrousel, no suma al article number
Ad de carrousel, no suma al article number
Ad de carrousel, no suma al article number
Ad de carrousel, no suma al article number
Ad de carrousel, no suma al article number
Ad de carrousel, no suma al article number
car_type: CUPRA - TERRAMAR
car_type: CUPRA - LEON
car_type: CUPRA - LEON
car_type: CUPRA - LEON
car_type: CUPRA - FORMENTOR
car_type: CUPRA - LEON
car_type: CUPRA - LEON
car_type: CUPRA - FORMENTOR
car_type: CUPRA - LEON
car_type: CUPRA - LEON
car_type: CUPRA - LEON
car_type: CUPRA - LEON
car_type: CUPRA - LEON
car_type: CUPRA - BORN
car_type: CUPRA - FORMENTOR
car_type: CUPRA - FORMENTOR
car_type: CUPRA - FORMENTOR
car_type: CUPRA - FORMENTOR
car_type: CUPRA LEON
car_type: CUPRA - LEON
car_type: CUPRA - ATECA
car_type: CUPRA - FORMENTOR
car_type: CUPRA - ATECA
car_type: CUPRA -

[]

In [89]:
df = pd.DataFrame(all_data)
df

,car_type,year,price,km,description,photo_url
0,TESLA - MODEL 3,2022,32.990 €,67.405 kms,Precio al contado: 32.990€ precio alternativo ...,https://images.milanuncios.com/api/v1/ma-ad-me...
1,TESLA - MODEL S 75D 4WD,2019,46.500 €,37.500 kms,Ocasionplus - 12000 coches - 100 centros ocasi...,https://images.milanuncios.com/api/v1/ma-ad-me...
2,TESLA - MODEL X 100D 4WD,2018,37.490 €,140.261 kms,Precio al contado: 37.490€ precio alternativo ...,https://images.milanuncios.com/api/v1/ma-ad-me...
3,TESLA - MODEL Y,2023,36.900 €,33.000 kms,"AÑO 2023 GARANTÍA TESLA Motor dual, tracción 4...",https://images.milanuncios.com/api/v1/ma-ad-me...
4,TESLA - MODEL Y,2023,44.990 €,29.000 kms,Interesados llamar al 680684213 -Único propiet...,https://images.milanuncios.com/api/v1/ma-ad-me...
...,...,...,...,...,...,...
2021,CUPRA - LEON,2025,28.885 €,0 kms,"Precio pago al contado: 28.885 €, consultar ca...",https://images.milanuncios.com/api/v1/ma-ad-me...
2022,CUPRA - FORMENTOR,2025,49.400 €,3.500 kms,Cupra formentor vz atendemos solo con cita pre...,https://images.milanuncios.com/api/v1/ma-ad-me...
2023,CUPRA - TAVASCAN,2025,42.500 €,0 kms,"Exclusiva oferta online en talleres rubens, co...",https://images.milanuncios.com/api/v1/ma-ad-me...
2024,CUPRA - LEON,2025,44.600 €,0 kms,"Exclusiva oferta online en talleres rubens, co...",https://images.milanuncios.com/api/v1/ma-ad-me...


In [78]:
df.isna().sum()

car_type       0
year           0
price          0
km             0
description    0
photo_url      0
dtype: int64

In [103]:
df['car_type'].value_counts().head(33)

car_type
TESLA - MODEL 3       223
CUPRA - FORMENTOR     180
CUPRA - LEON          117
PORSCHE - CAYENNE     111
AUDI - A3             105
AUDI - A4              87
BMW - X2               86
BMW - SERIE            86
TESLA - MODEL Y        75
PORSCHE - PANAMERA     72
TESLA - MODEL S        69
PORSCHE - TAYCAN       65
AUDI - A6              64
PORSCHE - 911          57
BMW - X1               52
PORSCHE - MACAN        42
TESLA - MODEL X        38
CUPRA - TERRAMAR       37
CUPRA - ATECA          36
AUDI - A5              35
BMW - X3               34
BMW - X5               33
BMW - COMPACT          24
PORSCHE - BOXSTER      23
CUPRA - BORN           21
BMW - X6               19
AUDI - TT              18
AUDI - A1              18
AUDI - Q3              17
BMW - Z4               17
AUDI - Q5              11
AUDI - Q7              11
PORSCHE - 718          10
Name: count, dtype: int64

In [81]:
conteo_cv = df['description'].str.contains(r'\bcv\b', case=False, na=False).sum()
print(f"Número de descripciones que contienen 'cv': {conteo_cv}")

Número de descripciones que contienen 'cv': 170


In [92]:
def procesar_car_type(valor):
    try:
        if ' - ' not in valor:
            partes = valor.split()
            if len(partes) > 1:
                # Insertar '-' después de la primera palabra
                valor = partes[0] + ' - ' + ' '.join(partes[1:])
            else:
                return valor  # Si solo hay una palabra, devolver tal cual

        marca, modelo = valor.split(' - ', 1)
        modelo_palabras = modelo.split()
        if marca.lower() == 'tesla':
            nuevo_modelo = ' '.join(modelo_palabras[:2])
        else:
            nuevo_modelo = modelo_palabras[0]
        return f"{marca} - {nuevo_modelo}"
    except Exception:
        return valor  # devuelve sin cambios si algo falla

df['car_type'] = df['car_type'].apply(procesar_car_type)

In [117]:
df

,car_type,year,price,km,description,photo_url
0,TESLA - MODEL 3,2022,32.990 €,67.405 kms,Precio al contado: 32.990€ precio alternativo ...,https://images.milanuncios.com/api/v1/ma-ad-me...
1,TESLA - MODEL S,2019,46.500 €,37.500 kms,Ocasionplus - 12000 coches - 100 centros ocasi...,https://images.milanuncios.com/api/v1/ma-ad-me...
2,TESLA - MODEL X,2018,37.490 €,140.261 kms,Precio al contado: 37.490€ precio alternativo ...,https://images.milanuncios.com/api/v1/ma-ad-me...
3,TESLA - MODEL Y,2023,36.900 €,33.000 kms,"AÑO 2023 GARANTÍA TESLA Motor dual, tracción 4...",https://images.milanuncios.com/api/v1/ma-ad-me...
4,TESLA - MODEL Y,2023,44.990 €,29.000 kms,Interesados llamar al 680684213 -Único propiet...,https://images.milanuncios.com/api/v1/ma-ad-me...
...,...,...,...,...,...,...
2021,CUPRA - LEON,2025,28.885 €,0 kms,"Precio pago al contado: 28.885 €, consultar ca...",https://images.milanuncios.com/api/v1/ma-ad-me...
2022,CUPRA - FORMENTOR,2025,49.400 €,3.500 kms,Cupra formentor vz atendemos solo con cita pre...,https://images.milanuncios.com/api/v1/ma-ad-me...
2023,CUPRA - TAVASCAN,2025,42.500 €,0 kms,"Exclusiva oferta online en talleres rubens, co...",https://images.milanuncios.com/api/v1/ma-ad-me...
2024,CUPRA - LEON,2025,44.600 €,0 kms,"Exclusiva oferta online en talleres rubens, co...",https://images.milanuncios.com/api/v1/ma-ad-me...


In [ ]:
df.to_csv('../../../data/raw/milanuncios_com.csv')